### updateECRDatastoreIrisID
This is the 2nd step to update the ECR datastore after receiving new MPI data from LAC, after updating the `person_id` in the `updateECRDatastorePersonID` notebook.

This notebook joins the ECR datastore (`ecr`) and Person table from the Master Patient Index (`person`) on `person_id` to update the ECR datastore's `iris_id` with the most up-to-date `external_person_id` in the `person` table.

In [ ]:
pip install --upgrade pip

In [ ]:
pip install psycopg2-binary azure-identity

In [ ]:
import psycopg2
from delta.tables import *
from pyspark.sql.functions import *
from azure.identity import DefaultAzureCredential

In [ ]:
storage_account_name = "$STORAGE_ACCOUNT"
ECR_DELTA_TABLE_FILE_PATH = f"abfss://delta-tables@{storage_account_name}.dfs.core.windows.net/ecr-datastore" # changed


# Set your Key Vault information
vault_name = "$KEY_VAULT"
KEY_VAULT_URL = f"https://{vault_name}.vault.azure.net"

vault_linked_service = "$KEY_VAULT_LINKED_SERVICE"
credential = DefaultAzureCredential()
db_password =  TokenLibrary.getSecret(vault_name,"mpi-db-password",vault_linked_service)

# Database connection parameters
DB_NAME = "DibbsMpiDB"
DB_USER = "postgres"
DB_HOST = "$MPI_DB_HOST"
DB_PORT = "5432"
DB_TABLE = "person"


# Connect to the database
conn = psycopg2.connect(
    dbname=DB_NAME,
    user=DB_USER,
    password=db_password,
    host=DB_HOST,
    port=DB_PORT
)

# Create a cursor
cur = conn.cursor()

# Execute the query to get the list of tables in the database
cur.execute(f"""
    SELECT person_id, external_person_id
    FROM {DB_TABLE};
""")

# Fetch the results
data = cur.fetchall()
data

# Close the cursor and connection
cur.close()
conn.close()


# Prep the MPI data for merging with ECR data 
columns=['person_id','external_person_id']
person = spark.createDataFrame(data = data, schema = columns)


# Load ecr Delta table
ecr = DeltaTable.forPath(spark,ECR_DELTA_TABLE_FILE_PATH)

# Update ecr data with `external_person_id` from MPI by joining on `person_id`
ecr.alias("ecr") \
  .merge(
    person.alias("mpi_person"),
    "ecr.person_id = mpi_person.person_id") \
  .whenMatchedUpdate(set = { "iris_id": "mpi_person.external_person_id", "iris_id_date_added": date_format(current_timestamp(), 'yyyy-MM-dd') }) \
  .execute()
